In [1]:
!pip install ydata_profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=9bbdc6302e85fe5cc55077f9be5567ebe3e1a78d89cb6f07d2eb7f5aa1364ba9
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from ydata_profiling import ProfileReport

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

inclusion = pd.read_csv('Financial_inclusion_dataset.csv')

In [4]:
inclusion.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [5]:
inclusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [6]:
inclusion.isnull().sum()

,0
country,0
year,0
uniqueid,0
bank_account,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0


In [7]:
label_encoder = preprocessing.LabelEncoder()
for i in inclusion.columns:
    if inclusion[i].dtype == 'object':
        inclusion[i]= label_encoder.fit_transform(inclusion[i])

In [8]:
inclusion.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,0,2018,0,1,0,1,3,24,0,5,2,3,9
1,0,2018,1111,0,0,0,5,70,0,1,4,0,4
2,0,2018,2222,1,1,1,5,26,1,3,3,5,9
3,0,2018,3333,0,0,1,5,34,0,1,2,2,3
4,0,2018,4444,0,1,0,8,26,1,0,3,2,5


In [9]:
x = inclusion.drop(columns=['uniqueid','bank_account'], axis=1).values
y = inclusion['bank_account'].values

In [10]:
scaler = StandardScaler()
x = scaler.fit_transform(x)
x

array([[-1.29944427,  1.20854126, -0.80006449, ..., -0.39296169,
         0.83389097,  1.15462805],
       [-1.29944427,  1.20854126, -0.80006449, ...,  1.60812733,
        -1.65519051, -0.46420749],
       [-1.29944427,  1.20854126,  1.24989924, ...,  0.60758282,
         2.49327861,  1.15462805],
       ...,
       [ 1.94351424,  1.20854126, -0.80006449, ...,  1.60812733,
         0.00419714,  0.50709384],
       [ 1.94351424,  1.20854126,  1.24989924, ..., -2.39405071,
         0.83389097,  1.15462805],
       [ 1.94351424,  1.20854126, -0.80006449, ...,  0.60758282,
         0.83389097,  0.18332673]])

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [13]:
y_pred = model.predict(x_test)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)

Accuracy: 0.8792773645058448
Confusion Matrix:
 [[3987   76]
 [ 492  150]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      4063
           1       0.66      0.23      0.35       642

    accuracy                           0.88      4705
   macro avg       0.78      0.61      0.64      4705
weighted avg       0.86      0.88      0.85      4705



In [14]:
!pip install pypickle

In [15]:
import pypickle
filepath = 'inclusion.pkl'
pypickle.save(filepath, model)

[pypickle] Pickle file saved: [inclusion.pkl]


True